# Data Cleaning and Preparation

In [ ]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

In [ ]:
string_data[0] = None
string_data.isnull()

### Filtering Out Missing Data

In [ ]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

In [ ]:
data[data.notnull()]

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data
cleaned

In [ ]:
data.dropna(how='all')

In [ ]:
data[4] = NA
data
data.dropna(axis=1, how='all')

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df
df.dropna()
df.dropna(thresh=2)

### Filling In Missing Data

In [ ]:
df.fillna(0)

In [ ]:
df.fillna({1: 0.5, 2: 0})

In [ ]:
_ = df.fillna(0, inplace=True)
df

In [ ]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df
df.fillna(method='ffill')
df.fillna(method='ffill', limit=2)

In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

## Data Transformation

### Removing Duplicates

In [ ]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

In [ ]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

In [ ]:
data.drop_duplicates(['k1', 'k2'], keep='last')

### Transforming Data Using a Function or Mapping

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [ ]:
lowercased = data['food'].str.lower()
lowercased
data['animal'] = lowercased.map(meat_to_animal)
data

In [ ]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

### Replacing Values

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

In [ ]:
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

In [ ]:
data.replace({-999: np.nan, -1000: 0})

### Renaming Axis Indexes

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [ ]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

In [ ]:
data.index = data.index.map(transform)
data

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

In [ ]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

### Discretization and Binning

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [ ]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

In [ ]:
cats.codes
cats.categories
pd.value_counts(cats)

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

In [ ]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

In [ ]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats
pd.value_counts(cats)

In [ ]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

### Detecting and Filtering Outliers

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

In [ ]:
col = data[2]
col[np.abs(col) > 3]

In [ ]:
data[(np.abs(data) > 3).any(1)]

In [ ]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

In [ ]:
np.sign(data).head()

### Permutation and Random Sampling

In [ ]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

In [ ]:
df
df.take(sampler)

In [ ]:
df.sample(n=3)

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

### Computing Indicator/Dummy Variables

In [ ]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'])

In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

In [ ]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

In [ ]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

In [ ]:
genres

In [ ]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [ ]:
gen = movies.genres[0]
gen.split('|')
dummies.columns.get_indexer(gen.split('|'))

In [ ]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [ ]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

In [ ]:
np.random.seed(12345)
values = np.random.rand(10)
values
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

## String Manipulation

### String Object Methods

In [ ]:
val = 'a,b,  guido'
val.split(',')

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

In [ ]:
first, second, third = pieces
first + '::' + second + '::' + third

In [ ]:
'::'.join(pieces)

In [ ]:
'guido' in val
val.index(',')
val.find(':')

In [ ]:
val.index(':')

In [ ]:
val.count(',')

In [ ]:
val.replace(',', '::')
val.replace(',', '')

### Regular Expressions
regex

In [1]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

### 객체= re.compile('패턴')

In [2]:
#공백기준으로 split
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [3]:
#매칭되는 목록
regex.findall(text)

['    ', '\t ', '  \t']

In [4]:
#매칭되는 목록
type(regex.findall(text))

list

###### r'...' : \\\\ 대신 \ 가능. \문자가 escape되는걸 막음

In [9]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob ro%%_-+b@gmail.com
Ryan ryan@yahoo.com
myu %_%3-3%+%@myu.onmicrosoft.myuc
"""

In [10]:
# pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
# pattern = '[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
pattern= '[\w.%+-]+@[\w.]+\w{2,4}'
# pattern= '.*'

##### reg객체= re.compile('정규표현식', flags= re.옵션)
reg객체.findall(데이터) : 모든 문자열. 많으면 오래 걸림

In [11]:
# re.IGNORECASE makes the regex case-insensitive
# regex = re.compile(pattern, flags=re.IGNORECASE)
regex1 = re.compile(pattern)#, flags=re.MULTILINE)

regex1.findall(text)

['dave@google.com',
 'steve@gmail.com',
 'ro%%_-+b@gmail.com',
 'ryan@yahoo.com',
 '%_%3-3%+%@myu.onmicrosoft.myuc']

In [12]:
type(regex1.findall(text))

list

##### 제대로 반영이 되고 있는지 확인해보기
하나만 보자

In [127]:
m = regex1.search(text)
m
text[m.start():m.end()]

'dave@google.com'

##### 패턴이 문자열의 시작점부터 일치하는지?
아니면 None

###### 있으면 match객체 반환
pattern= '.*' 

#< _sre.SRE_Match object; span=(0, 20), match='Dave dave@google.com'>

In [128]:
print(regex1.match(text))

None


In [129]:
# regex1.match?

#### reg객체.sub('바꾼결과', 대상데이터)

In [141]:
from pandas import Series, DataFrame
import pandas as pd

In [110]:
print(regex1.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED
myu REDACTED



In [130]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex1 = re.compile(pattern, flags=re.IGNORECASE)

In [131]:
m = regex1.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [134]:
# m.groups?
# m.group?

In [145]:
m.groups(2)

('wesm', 'bright', 'net')

In [151]:
type(m.groups(2))

tuple

In [144]:
m.group(0)

'wesm@bright.net'

In [152]:
type(m.group(0))

str

In [147]:
m.group(1)

'wesm'

In [148]:
m.group(2)

'bright'

In [149]:
m.group(3)

'net'

In [113]:
regex1.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('ro%%_-+b', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com'),
 ('%_%3-3%+%', 'myu.onmicrosoft', 'myuc')]

In [142]:
DataFrame(regex1.findall(text))

,0,1,2
0,dave,google,com
1,steve,gmail,com
2,ro%%_-+b,gmail,com
3,ryan,yahoo,com
4,%_%3-3%+%,myu.onmicrosoft,myuc


In [114]:
print(regex1.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: ro%%_-+b, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com
myu Username: %_%3-3%+%, Domain: myu.onmicrosoft, Suffix: myuc



## 예제_정규식
홈페이지주소만 뽑기

1. reg객체= re.compile('정규표현식')#, flags= re.옵션)
2. reg객체.search(데이터) : 하나만 뽑아보기
3. reg객체.findall(데이터) : 모든 문자열. 많으면 오래 걸림

In [244]:
from pandas import DataFrame
import pandas as pd

In [171]:
with open('data/shoppingmall.txt','r') as f:
    lines= f.readlines()

In [172]:
# lines= str(lines).replace('\\n','').replace(',','').replace('\'','')
# lines_str= str(lines)
# lines_str= ''.join(lines)
# lines_str= re.sub('\n','',''.join(lines))
lines_str= re.sub('[\n ]','',''.join(lines)) #완벽한 정규식을 위해 에러만들려고

In [173]:
lines_str

'검색광고 사이트 >등록 1. 갓성비호갱방지 슈퍼스타아이갓성비호갱방지 슈퍼스타아이 네이버페이    이 가격 실화냐? 응 실화야! 같은 옷, 같은 소재 가격비교 호갱방지 슈퍼스타아이http://www.superstari.co.kr 광고집행기간  61개월 이상 2. 옥션 의류쇼핑몰옥션 의류쇼핑몰 카메라, 캠코더, 초특가 세일 중! 의류쇼핑몰, 10%쿠폰+제휴카드 추가할인http://www.auction.co.kr 광고집행기간  61개월 이상 3. 국가대표 판촉물 한시루국가대표 판촉물 한시루 No.1판촉물, 품질보장, 업종별BEST상품제안, 무료인쇄, 무료배송http://www.hansirulife.com 광고집행기간  13~24개월 4. 설레는 봄코디 완성, 메롱샵설레는 봄코디 완성, 메롱샵 네이버페이    최대 90%할인받아 봄쇼핑하자! 매일매일 폭풍업뎃! 신상5%, 봄에는 역시 메롱샵http://www.merongshop.com 광고집행기간  61개월 이상 5. 쉽고빠른 도매, 2002도매쉽고빠른 도매, 2002도매 원가이하 남/여 의류도매몰, 동대문에서 볼 수 없는 의류쇼핑몰 가격.http://www.2002.co.kr 광고집행기간  37~60개월 6. SNS 쇼핑몰은 페이링크 SNS결제, 블로그결제, 모바일 쇼핑몰까지! 페이링크http://www.o2ocommerce.kr/ 광고집행기간  0~3개월 7. 압도적 재구매율! 리-판촉압도적 재구매율! 리-판촉 판촉물, 답례품, 사은품, 기념품, 업종별 행사별 의류쇼핑몰 다양하게 구비http://www.leegift.co.kr/ 광고집행기간  4~12개월 8. 의류 인기쇼핑몰 서아몰 네이버페이    매일 매일 새로운 옷 업데이트/아동복/여성복/남성복/특가 할인/데일리 룩http://smartstore.naver.com/wowow312 광고집행기간  4~12개월 9. 화이트데이 코디는 멋남에서!화이트데이 코디는 멋남에서! 네이버페이    여심저격 봄코디 할인 중! 코디세트 조기품절 주의, 1+1 봄신상가득 멋남ht

### 주의 ( )
( ) 치지 않은 부분에 대해서는 매칭을 해도 findall 결과에 나오지 않음
> 마치 look ahead 처럼

In [256]:
# reg_link= re.compile('https?://[\dA-Za-z/.-]+')
# reg_link= re.compile('https?://([\dA-Za-z/.-]+)')
# reg_link= re.compile('(?:https?://)([\dA-Za-z.-]+)([/\dA-Za-z.-]+)?')
reg_link= re.compile('(https?://)([\dA-Za-z.-]+)([/\dA-Za-z.-]+)?')

In [257]:
# test= reg_link.search(lines_str)
# text[test.start():test.end()]

In [258]:
links1= reg_link.findall(lines_str)

In [259]:
print(len(links1))
print(type(links1))
links1

25
<class 'list'>


[('http://', 'www.superstari.co.kr', ''),
 ('http://', 'www.auction.co.kr', ''),
 ('http://', 'www.hansirulife.com', ''),
 ('http://', 'www.merongshop.com', ''),
 ('http://', 'www.2002.co.kr', ''),
 ('http://', 'www.o2ocommerce.kr', '/'),
 ('http://', 'www.leegift.co.kr', '/'),
 ('http://', 'smartstore.naver.com', '/wowow312'),
 ('http://', 'www.mutnam.com', ''),
 ('http://', 'auction.co.kr', ''),
 ('http://', 'ddrago.co.kr', ''),
 ('http://', 'www.11st.co.kr', ''),
 ('http://', 'www.jdcdutyfree.com', '/'),
 ('http://', 'ATTRANGS.co.kr', ''),
 ('http://', 'www.minibbong.co.kr', ''),
 ('http://', 'www.ottaku.co.kr', '/'),
 ('http://', 'www.starguje.com', '/'),
 ('http://', 'tqdc.co.kr', '/'),
 ('http://', 'www.gmarket.co.kr', ''),
 ('http://', 'dmantic.com', ''),
 ('http://', 'www.gaenso.com', ''),
 ('http://', 'www.secretlabel.co.kr', ''),
 ('http://', 'www.wemakeprice.com', ''),
 ('http://', 'www.dartz-smc.co.kr', ''),
 ('http://', 'www.gsshop.com', '')]

In [246]:
DataFrame(links1, columns= ['protocol','domain','directory'])

,protocol,domain,directory
0,http://,www.superstari.co.kr,
1,http://,www.auction.co.kr,
2,http://,www.hansirulife.com,
3,http://,www.merongshop.com,
4,http://,www.2002.co.kr,
5,http://,www.o2ocommerce.kr,/
6,http://,www.leegift.co.kr,/
7,http://,smartstore.naver.com,/wowow312
8,http://,www.mutnam.com,
9,http://,auction.co.kr,


# ?????

In [239]:
links1= reg_link.findall(lines_str)
links2= reg_link.match(lines_str)

In [260]:
links1.groups()

AttributeError: 'list' object has no attribute 'groups'

In [210]:
re.sub('(?=ㅋ)ㅋ+','','ㅋㅋㅋㅋㅋㅋ')

''

### Vectorized String Functions in pandas

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data
data.isnull()

In [ ]:
data.str.contains('gmail')

In [ ]:
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

In [ ]:
matches.str.get(1)
matches.str[0]

In [ ]:
data.str[:5]

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion